In [1]:
"""
We use following lines because we are running on Google Colab
If you are running notebook on a local computer, you don't need this cell
"""
from google.colab import drive
drive.mount('/content/gdrive')
import os
os.chdir('/content/gdrive/My Drive/finch/tensorflow1/free_chat/chinese/main')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [0]:
%tensorflow_version 1.x

In [3]:
import tensorflow as tf
import time

from pathlib import Path

print("TensorFlow Version", tf.__version__)
print('GPU Enabled:', tf.test.is_gpu_available())

TensorFlow Version 1.15.0
GPU Enabled: False


In [0]:
def get_vocab(f_path):
  word2idx = {}
  with open(f_path) as f:
    for i, line in enumerate(f):
      line = line.rstrip('\n')
      word2idx[line] = i
  return word2idx

def parse_fn(text):
  parse = lambda text: [[params['char2idx'].get(c, len(params['char2idx'])) for c in list(text)]]
  return {'words': parse(text)}

In [5]:
params = {'export_dir': '../model/lstm_seq2seq_export',}

params['char2idx'] = get_vocab('../vocab/char.txt')

params['idx2char'] = {idx: char for char, idx in params['char2idx'].items()}

subdirs = [x for x in Path(params['export_dir']).iterdir()
           if x.is_dir() and 'temp' not in str(x)]
latest = str(sorted(subdirs)[-1])

predict_fn = tf.contrib.predictor.from_saved_model(latest)

The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

Instructions for updating:
This function will only be available through the v1 compatibility library as tf.compat.v1.saved_model.loader.load or tf.compat.v1.saved_model.load. There will be a new function for importing SavedModels in Tensorflow 2.0.
INFO:tensorflow:Restoring parameters from ../model/lstm_seq2seq_export/1574746280/variables/variables


In [0]:
while True:
  text_inp = input('Input:')
  t0 = time.time()
  predictions = predict_fn(parse_fn(text_inp))
  print('Output Shape:', predictions['output'].shape)
  for j in range(3):
    print('A{}:'.format(j+1), ' '.join([params['idx2char'].get(idx, len(params['idx2char'])) for idx in predictions['output'][0, :, j] if (idx != 0 and idx != 2)]))
  print('Spent %.2f sec' % (time.time() - t0))
  print()

Input:你好
Output Shape: (1, 18, 3)
A1: 主 人 你 好 O ( ∩ _ ∩ ) O
A2: 你 好 呀
A3: 你 好 主 人 ， 你 好
Spent 0.67 sec

Input:早上好
Output Shape: (1, 20, 3)
A1: 早 上 好 O ( ∩ _ ∩ ) O
A2: 早 上 好
A3: 主 人 早 上 好 ！
Spent 0.22 sec

Input:晚上好
Output Shape: (1, 10, 3)
A1: 晚 上 好
A2: 晚 上 好 好
A3: 晚 上 好 ~
Spent 0.11 sec

Input:再见
Output Shape: (1, 10, 3)
A1: 再 见 ！
A2: 再 见 亲 爱 的 主 淫
A3: 再 见
Spent 0.11 sec

Input:好久不见
Output Shape: (1, 50, 3)
A1: 好 久 不 见
A2: 好 久 不 见 [ 嘻 嘻 ]
A3: 好 久 不 见 ~
Spent 0.71 sec

Input:想死你了
Output Shape: (1, 50, 3)
A1: 我 也 想 死 你 了
A2: 我 也 想 死 你
A3: 想 死 你 了
Spent 0.53 sec

Input:谢谢你
Output Shape: (1, 14, 3)
A1: 不 要 谢 谢 你
A2: 谢 谢 你
A3: 不 要 谢 谢 你 了
Spent 0.21 sec

Input:爱你
Output Shape: (1, 14, 3)
A1: 我 也 爱 你
A2: 我 也 爱 你   主 人 〜 ( ^ ○ ^ )
A3: 我 也 爱 你 ！ ！ 吼 吼 ！ ！
Spent 0.20 sec

Input:哈啰
Output Shape: (1, 11, 3)
A1: 哈 啰
A2: 哈 啰 。
A3: 哈 啰 哈 啰
Spent 0.18 sec

Input:你叫什么名字
Output Shape: (1, 19, 3)
A1: 你 叫 什 么
A2: 你 叫 什 么 我 就 叫 什 么
A3: 你 叫 什 么 ， 我 就 叫 什 么
Spent 0.29 sec

Input:你几岁了
Output Shape: (1, 33, 